This presents the exploration of the superstore sales data in SQL. I will attempt to check if there is any data cleaning necessary
The source of data is Kaggle: [https://www.kaggle.com/blurredmachine/superstore-time-series-dataset](https://www.kaggle.com/blurredmachine/superstore-time-series-dataset)
I am only displaying the top 5 rows in each table for easier viewing in this notebook
Key tasks inspired by projectpro.io
- Use the LEAD window function to create a new column sales\_next that displays the sales of the next row in the dataset. This function will help you quickly compare a given row’s values and values in the next row.
    
- Create a new column sales\_previous to display the values of the row above a given row.
    
- Rank the data based on sales in descending order using the RANK function.
    
- Use common SQL commands and aggregate functions to show the monthly and daily sales, discount, and quantity averages.
    
- Evaluate moving averages using the window functions

In [1]:
select top (5) *
from dbo.superstore

(5 rows affected)

Total execution time: 00:00:04.548

Row_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Customer_Name,Segment,City,State,Country,Market,Region,Product_ID,Category,Sub_Category,Product_Name,Sales,Quantity,Discount,Profit,Shipping_Cost,Order_Priority
42433,AG-2011-2040,2011-01-01,2011-01-06,Standard Class,TB-11280,Toby Braunhardt,Consumer,Constantine,Constantine,Algeria,Africa,Africa,OFF-TEN-10000025,Office Supplies,Storage,"Tenex Lockers, Blue",408.3,2,0,106.14,35.46,Medium
22253,IN-2011-47883,2011-01-01,2011-01-08,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,Australia,APAC,Oceania,OFF-SU-10000618,Office Supplies,Supplies,"Acme Trimmer, High Speed",120.366,3,0.1,36.036,9.72,Medium
48883,HU-2011-1220,2011-01-01,2011-01-05,Second Class,AT-735,Annie Thurman,Consumer,Budapest,Budapest,Hungary,EMEA,EMEA,OFF-TEN-10001585,Office Supplies,Storage,"Tenex Box, Single Width",66.12,4,0,29.64,8.17,High
11731,IT-2011-3647632,2011-01-01,2011-01-05,Second Class,EM-14140,Eugene Moren,Home Office,Stockholm,Stockholm,Sweden,EU,North,OFF-PA-10001492,Office Supplies,Paper,"Enermax Note Cards, Premium",44.865,3,0.5,-26.055,4.82,High
22255,IN-2011-47883,2011-01-01,2011-01-08,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,Australia,APAC,Oceania,FUR-FU-10003447,Furniture,Furnishings,"Eldon Light Bulb, Duo Pack",113.67,5,0.1,37.77,4.7,Medium


2\. I want to check that there is no null data in sales, discount, and quantity that we plan to aggregate.

There is no null data

In [5]:
select top (5) sales,discount,quantity
from superstore
where sales is null or discount is null or quantity is null

(0 rows affected)

Total execution time: 00:00:00.016

sales,discount,quantity


3. I am using statistical description to check for any suspicious data in sales, quantity, and discount.

In [6]:
select top (5) avg(sales) as AvgSales,avg(discount) as AvgDisc,avg(quantity) as AvgQty,
max(sales) as MaxSales,max(discount) as MaxDisc,avg(quantity) as MaxQty,
Min(sales) as MinSales,Min(discount) as MinDisc,Min(quantity) as MinQty
from superstore



(1 row affected)

Total execution time: 08:18:38.966

AvgSales,AvgDisc,AvgQty,MaxSales,MaxDisc,MaxQty,MinSales,MinDisc,MinQty
246.17664597849006,0.14281162737469605,3,22638.48,0.85,3,0.444,0,1


Nothing suspicious

4\. Use the LEAD window function to create a new column sales\_next that displays the sales of the next row in the dataset. 

I decided to partition by product name so that I can compare current and next sales of the same product. It helps to identify changes in sales i.e. product with a single sale entry

In [6]:
select top (5) product_name, sales, LEAD(sales,1) over(partition by product_name order by product_name) as sales_next
from superstore
order by product_name asc

(5 rows affected)

Total execution time: 00:00:00.125

product_name,sales,sales_next
#6 3/4 Gummed Flap White Envelopes,15.840000000000002,NULL
"1/4 Fold Party Design Invitations & White Envelopes, 24 8-1/2"" X 11"" Cards, 25 Env./Pack",5.88,44.099999999999994
"#10- 4 1/8"" x 9 1/2"" Security-Tint Envelopes",22.919999999999998,NULL
"#10 White Business Envelopes,4 1/8 x 9 1/2",109.69,125.36
"#10 White Business Envelopes,4 1/8 x 9 1/2",125.36,NULL


5. Create a new column sales\_previous to display the values of the row above a given row using lag window funtion.

In [2]:
select top (5) product_name, sales, LAG(sales,1) over(partition by product_name order by product_name) as sales_previos
from dbo.superstore
order by product_name asc

(5 rows affected)

Total execution time: 00:02:38.685

product_name,sales,sales_previos
"""While you Were Out"" Message Book, One Form per Page",8.904,NULL
"""While you Were Out"" Message Book, One Form per Page",8.904,8.904
"""While you Were Out"" Message Book, One Form per Page",7.42,8.904
"#10- 4 1/8"" x 9 1/2"" Recycled Envelopes",13.984,NULL
"#10- 4 1/8"" x 9 1/2"" Recycled Envelopes",48.944,13.984


6. Rank the data based on sales in descending order using the RANK function. In this example, I have decided to partition by category so I can see the product in a categry with the most sales.

In [3]:
select top (5) category, product_name, sales, RANK() OVER (PARTITION BY category ORDER BY sales DESC) AS SalesRankCat
from superstore
order by category desc

(5 rows affected)

Total execution time: 00:00:11.153

category,product_name,sales,SalesRankCat
Technology,Cisco TelePresence System EX90 Videoconferencing Unit,22638.48,1
Technology,Canon imageCLASS 2200 Advanced Copier,17499.95,2
Technology,Canon imageCLASS 2200 Advanced Copier,17499.95,2
Technology,Canon imageCLASS 2200 Advanced Copier,13999.96,4
Technology,Canon imageCLASS 2200 Advanced Copier,11199.968,5


7\. Rank the data based on sales in descending order using the RANK function. In this example, I will rank all the product's sales in a single list

In [9]:
select top (5) product_name, sales, RANK() OVER (ORDER BY sales DESC) AS SalesRank
from superstore
order by sales desc

(5 rows affected)

Total execution time: 00:00:00.073

product_name,sales,SalesRank
Canon imageCLASS 2200 Advanced Copier,13999.96,1
GBC Ibimaster 500 Manual ProClick Binding System,9892.74,2
"HP Designjet T520 Inkjet Large Format Printer - 24"" Color",8749.95,3
Fellowes PB500 Electric Punch Plastic Comb Binding Machine with Manual Bind,5083.96,4
Canon PC1060 Personal Laser Copier,4899.93,5


8\. Use common SQL commands and aggregate functions to show the daily sales averages.

In [10]:
select top (5) order_date,avg(sales) as DailyAvgSales, avg(discount) as DailyAvgDiscount, avg(quantity) as DailyAvgQty
from superstore
group by order_date
order by order_date asc

(5 rows affected)

Total execution time: 00:00:00.037

order_date,DailyAvgSales,DailyAvgDiscount,DailyAvgQty
2014-01-13,20.423000000000002,0.44999999999999996,4
2014-01-21,25.248,0.2,3
2014-01-23,23.009999999999998,0,4
2014-01-26,76.32000000000001,0,2
2014-02-02,99.64800000000001,0.44999999999999996,3


9\. Use common SQL commands and aggregate functions to show the monthly average sales, quantity, and discount.

In [11]:
-- create a temporary table
drop table if exists TempSales3
create table TempSales3(order_date date, monthly varchar(255), sales numeric, quantity numeric, discount numeric)
insert into TempSales3
--covert order date to MMMYYYY
Select order_date, replace(right(CONVERT(Varchar(255),order_date,106), 8), ' ', '-') as Monthly,sales, quantity, discount
from superstore
order by order_date asc

select top (5) convert(date,concat('01-',monthly),106) as Order_Date1,
avg(sales) as MonthlySales
, avg(quantity) as MonthlyQty
, avg(discount) as MonthlyDiscount 
from TempSales3
group by monthly 
order by Order_Date1


(1993 rows affected)

(5 rows affected)

Total execution time: 00:00:00.186

Order_Date1,MonthlySales,MonthlyQty,MonthlyDiscount
2014-01-01,37.714285,3.571428,0.142857
2014-02-01,79.000000,3.157894,0.105263
2014-03-01,176.111111,3.444444,0.027777
2014-04-01,231.346153,3.846153,0.000000
2014-05-01,258.875000,3.500000,0.125000


10\. I am going to use the same aggregate function to create yearly average sales, quantity, and discount

In [6]:
-- create a temporary table
drop table if exists TempSales4
create table TempSales4(order_date date, yearly varchar(255), sales numeric, quantity numeric, discount numeric)
insert into TempSales4
--covert order date to MMMYYYY
Select order_date, right(CONVERT(Varchar(255),order_date,106), 4) as yearly,sales, quantity, discount
from superstore
order by order_date asc

select yearly, 
avg(sales) as YearlySales
, avg(quantity) as YearlyQty
, avg(discount) as YearlyDiscount 
from TempSales4
group by yearly 
order by yearly

(65535 rows affected)

(4 rows affected)

Total execution time: 00:00:28.859

yearly,YearlySales,YearlyMonthlyQty,YearlyDiscount
2011,251.491432,3.501713,0.118354
2012,242.236257,3.467582,0.112901
2013,247.120458,3.489068,0.108172
2014,244.864081,3.460258,0.112385


10. Rolling average and rolling total of sales, and quantity

In [2]:

select top (5) row_ID,
avg(sales) over(order by row_ID) as RollingAvgSales,
avg(quantity) over(order by row_ID) as RollingAvgQty,
--avg(discount) over(order by order_date) as RollingAvgDiscount,
--total
sum(sales) over(order by row_ID) as RollingTotalSales,
sum(quantity) over(order by row_ID) as RollingTotalQty
--sum(discount) over(order by order_date) as RollingTotalDiscount
from superstore
order by row_ID


(5 rows affected)

Total execution time: 00:02:07.669

row_ID,RollingAvgSales,RollingAvgQty,RollingTotalSales,RollingTotalQty
1,13.08,3,26.16,6
1,13.08,3,26.16,6
2,92.77333333333333,4,278.32,14
3,117.9,4,471.6,16
4,101.408,4,507.04,20
